In [3]:
!pip install librosa

  Using cached numba-0.59.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.7 kB)
  Using cached pooch-1.8.1-py3-none-any.whl.metadata (9.5 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
  Using cached llvmlite-0.42.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 4.9 MB/s eta 0:00:00a 0:00:01
Using cached lazy_loader-0.4-py3-none-any.whl (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 9.2 MB/s eta 0:00:00
Using cached numba-0.59.1-cp312-cp312-macosx_11_0_arm64.whl (2.6 MB)
Using cached pooch-1.8.1-py3-none-any.whl (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.2/390.2 kB 6.1 MB/s eta 0:00:00ta 0:00:01
Using cached llvmlite-0.42.0-cp312-cp312-macosx_11_0_arm64.whl (28.8 MB)


In [20]:
import pandas as pd
import numpy as np
import librosa

# Read the Feather file
df = pd.read_feather('CRSP_daily_data_for_project(Technical_Analysis).feather')
# df = df[(df['PERMNO']==10000) |( df['PERMNO']==10001)|(df['PERMNO']==10002)| (df['PERMNO']==93434)|(df['PERMNO']==93435)|(df['PERMNO']==93436)]


In [21]:
df[df['PERMNO']==10000]

,PERMNO,date,Open,High,Low,Close,Volume,SHROUT,vwretx,ewretx,sprtrn
1,10000,1986-01-07,NaN,2.750,2.3750,2.56250,1000.0,3680.0,0.013800,0.011046,0.014954
2,10000,1986-01-08,NaN,2.625,2.3750,2.50000,12800.0,3680.0,-0.020750,-0.005135,-0.027268
3,10000,1986-01-09,NaN,2.625,2.3750,2.50000,1400.0,3680.0,-0.011315,-0.011659,-0.008944
4,10000,1986-01-10,NaN,2.625,2.3750,2.50000,8500.0,3680.0,0.000047,0.003632,-0.000728
5,10000,1986-01-13,NaN,2.750,2.5000,2.62500,5450.0,3680.0,0.002680,0.002369,0.003690
...,...,...,...,...,...,...,...,...,...,...,...
359,10000,1987-06-08,NaN,0.250,0.1875,0.21875,0.0,3893.0,0.008563,0.001508,0.011143
360,10000,1987-06-09,NaN,0.250,0.1875,0.21875,0.0,3893.0,0.001918,0.002217,0.001887
361,10000,1987-06-10,NaN,0.250,0.1875,0.21875,0.0,3893.0,0.001492,0.001049,0.000639
362,10000,1987-06-11,NaN,0.250,0.1875,0.21875,500.0,3893.0,0.003427,0.002650,0.004236


In [22]:
from datetime import datetime
df['date']=df['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

In [25]:
df['date'].max()

Timestamp('2023-12-29 00:00:00')

In [23]:
# Function: Generate CQT data for each stock
def create_cqt_df(stock_df, n_bins=20):
    stock_returns = stock_df['vwretx'].values

    # Data preprocessing: Ensure all values are finite
    stock_returns = np.nan_to_num(stock_returns, nan=0.0, posinf=0.0, neginf=0.0)

    sr = 1
    fmin = 0.01  # Minimum frequency
    hop_length = 1
    cqt_result = librosa.cqt(stock_returns, n_bins=n_bins, sr=sr, hop_length=hop_length, fmin=fmin)
    cqt_result_db = librosa.amplitude_to_db(np.abs(cqt_result), ref=np.mean)

    cqt_df = pd.DataFrame(cqt_result_db.T, columns=[f'CQT_{i+1}' for i in range(cqt_result_db.shape[0])])
    cqt_df = cqt_df.loc[:len(stock_df)-1, :]  # Ensure the length matches the original data
    cqt = pd.concat([stock_df['date'], cqt_df], axis=1)
    return cqt

# Function: Create rolling windows and retain date indices
def create_rolling_windows_with_dates(data, cqt_window_size, date_to_index):
    windows = []
    date_indices = []
    for i in range(len(data) - cqt_window_size + 1):
        window = data.iloc[i:i + cqt_window_size, 1:].values  # Exclude the date column, keep only CQT data
        windows.append(window)
        start_date = data.iloc[i]['date']
        if start_date in date_to_index:
            date_indices.append(date_to_index[start_date])
    return np.array(windows), date_indices


In [92]:
def get_pre_processed_data(df, start_date, end_date, window_size=30):
    
    df_sub = df[(df['date']>=start_date) & (df['date']<=end_date)]

    # Get the list of all stock codes
    stock_codes = set(df_sub['PERMNO'].unique())
    # stock_codes = set()

    cqt_window_size = window_size
    n_bins = 20  # Number of CQT bins

    # Get all unique dates
    unique_dates = df_sub['date'].unique()

    # drop the PERMNO for which data is not available on all unique_dates
    for _,r in df_sub.groupby(['date']):
        stock_codes.intersection_update(set(r['PERMNO'].to_list()))

    stock_codes = list(stock_codes)[:10]
    
    df_sub = df_sub[df_sub['PERMNO'].isin(stock_codes)]
    
    # Create the 4D array, initialized with NaN
    num_dates = len(unique_dates)
    max_shape = (len(stock_codes), num_dates - cqt_window_size + 1, cqt_window_size, n_bins)
    final_4d_array = np.full(max_shape, np.nan)

    # Create the date-to-index mapping
    date_to_index = {date: idx for idx, date in enumerate(unique_dates)}

    # Create the stock code-to-index mapping
    stock_code_to_index = {stock_code: idx for idx, stock_code in enumerate(stock_codes)}

    # Iterate over each stock and fill the 4D array
    for stock_code in stock_codes:
        stock_idx = stock_code_to_index[stock_code]
        stock_data = df_sub[df_sub['PERMNO'] == stock_code].reset_index(drop=True)
        
        # Generate CQT data
        cqt_data = create_cqt_df(stock_data, n_bins)

        # Create rolling windows and retain date indices
        rolling_windows_3d, date_indices = create_rolling_windows_with_dates(cqt_data, cqt_window_size, date_to_index)
        
        # Fill the 4D array
        for window_idx, (window_data, date_idx) in enumerate(zip(rolling_windows_3d, date_indices)):
            if window_data.shape == (cqt_window_size, n_bins):
                final_4d_array[stock_idx, date_idx, :, :] = window_data

    df_sub = df_sub.sort_values(by=['PERMNO', 'date'])
    df_sub['Ret'] = df_sub.groupby('PERMNO')['Close'].apply(lambda x: x.shift(-30).pct_change()).reset_index(level=0, drop=True)

    # df_sub['Ret'] = df_sub.groupby('PERMNO')['Close'].apply(lambda x: x.shift(-30).pct_change())

    return final_4d_array, df_sub, stock_codes

In [93]:
start_date = datetime.strptime("2000-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2001-01-01", "%Y-%m-%d")

In [94]:
import numpy as np
final_4d_array, df_sub, stock_codes = get_pre_processed_data(df, start_date, end_date)

/Users/tush/miniconda3/envs/finance/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=252
  warnings.warn(
/Users/tush/miniconda3/envs/finance/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=252
  warnings.warn(
/Users/tush/miniconda3/envs/finance/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=252
  warnings.warn(
/Users/tush/miniconda3/envs/finance/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=252
  warnings.warn(
/Users/tush/miniconda3/envs/finance/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=252
  warnings.warn(
/Users/tush/miniconda3/envs/finance/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserW

In [95]:
df_sub

,PERMNO,date,Open,High,Low,Close,Volume,SHROUT,vwretx,ewretx,sprtrn,Ret
27160016,32791,2000-01-03,39.1250,39.2500,36.000,36.0625,612200.0,108088.0,-0.006810,0.002860,-0.009549,NaN
27160017,32791,2000-01-04,35.1250,37.7500,34.875,35.7500,894600.0,108088.0,-0.039679,-0.017486,-0.038345,0.040580
27160018,32791,2000-01-05,35.2500,37.0000,35.000,36.5000,622900.0,108088.0,-0.001009,0.007743,0.001922,-0.027855
27160019,32791,2000-01-06,36.0000,37.8750,35.875,37.6250,1440599.0,108088.0,-0.007547,0.004453,0.000956,-0.057307
27160020,32791,2000-01-07,37.9375,39.0000,37.625,38.6250,984900.0,108088.0,0.032514,0.016991,0.027090,-0.018237
...,...,...,...,...,...,...,...,...,...,...,...,...
47957970,65700,2000-12-22,4.6250,4.6250,4.500,4.5625,500.0,3982.0,0.030141,0.026251,0.024403,0.000000
47957971,65700,2000-12-26,4.5625,4.5625,4.250,4.5000,21600.0,3982.0,0.005514,-0.005825,0.007060,0.000000
47957972,65700,2000-12-27,4.5000,4.5000,4.500,4.5000,200.0,3982.0,0.013651,0.010334,0.010440,0.000000
47957973,65700,2000-12-28,4.5000,4.5000,4.500,4.5000,7888.0,3982.0,0.009377,0.022075,0.003988,0.000000


In [97]:
df_sub[df_sub['PERMNO']==32791]

,PERMNO,date,Open,High,Low,Close,Volume,SHROUT,vwretx,ewretx,sprtrn,Ret
27160016,32791,2000-01-03,39.1250,39.2500,36.0000,36.0625,612200.0,108088.0,-0.006810,0.002860,-0.009549,NaN
27160017,32791,2000-01-04,35.1250,37.7500,34.8750,35.7500,894600.0,108088.0,-0.039679,-0.017486,-0.038345,0.040580
27160018,32791,2000-01-05,35.2500,37.0000,35.0000,36.5000,622900.0,108088.0,-0.001009,0.007743,0.001922,-0.027855
27160019,32791,2000-01-06,36.0000,37.8750,35.8750,37.6250,1440599.0,108088.0,-0.007547,0.004453,0.000956,-0.057307
27160020,32791,2000-01-07,37.9375,39.0000,37.6250,38.6250,984900.0,108088.0,0.032514,0.016991,0.027090,-0.018237
...,...,...,...,...,...,...,...,...,...,...,...,...
27160263,32791,2000-12-22,45.0000,45.5000,44.3125,45.0000,749600.0,109890.0,0.030141,0.026251,0.024403,0.000000
27160264,32791,2000-12-26,45.0000,47.9375,45.0000,47.9375,733400.0,109890.0,0.005514,-0.005825,0.007060,0.000000
27160265,32791,2000-12-27,47.9375,48.1250,46.0000,47.1875,850400.0,109890.0,0.013651,0.010334,0.010440,0.000000
27160266,32791,2000-12-28,47.4375,49.0625,46.8750,49.0625,823900.0,109890.0,0.009377,0.022075,0.003988,0.000000


In [98]:
stock_codes

[65541, 32791, 32803, 65584, 32839, 32870, 65656, 32897, 65672, 65700]

In [62]:
final_4d_array.shape

(10, 223, 30, 20)

In [38]:
!pip install keras tensorflow

  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.16.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 MB 5.4 MB/s eta 0:00:0000:0100:01
Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 4.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 3.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.6/393.6 kB 6.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.5 MB/s eta 0:00:00
Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl (394 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [66]:
stock_codes

[65541, 32791, 32803, 65584, 32839, 32870, 65656, 32897, 65672, 65700]

In [56]:
# Apply LSTM model to the CQT data
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [111]:
def apply_lstm_model(final_4d_array, df_sub, stock_codes):
    model = Sequential()
    model.add(LSTM(50, input_shape=(final_4d_array.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    for stock in range(final_4d_array.shape[0]):
        stock_data = final_4d_array[stock]
        ret = df_sub[df_sub.PERMNO == stock_codes[stock]]['Ret'].values[-stock_data.shape[0]:].reshape(-1, 1)
        # Normalize the data
        scaler = MinMaxScaler()
        stock_data = scaler.fit_transform(stock_data.reshape(stock_data.shape[0], stock_data.shape[1] * stock_data.shape[2]))
        print(stock_data.shape, ret.shape)
        # Split the data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(stock_data, ret, test_size=0.2, shuffle=False)

        # Fit the model
        model.fit(X_train, y_train, epochs=100, batch_size=32)

        # Evaluate the model
        train_loss = model.evaluate(X_train, y_train, verbose=0)
        test_loss = model.evaluate(X_test, y_test, verbose=0)
        print(f'Train Loss: {train_loss}')
        print(f'Test Loss: {test_loss}')

    return model

In [112]:
model = apply_lstm_model(final_4d_array, df_sub, stock_codes)
model.summary()

(223, 600) (223, 1)
Epoch 1/100


/Users/tush/miniconda3/envs/finance/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - loss: 0.0107
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 0.0052
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - loss: 0.0037
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 0.0027
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 0.0023
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - loss: 0.0023
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - loss: 0.0017
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 0.0016
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 0.0017
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 0.0016
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.0013
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 0.0012
Epoch 13/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.0012
Epoch 14/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 0.0015
Epoch 15/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 9.8665e-04
Epoch 16/100
6/6 ━━━━━━━━━━

KeyboardInterrupt: 

In [ ]:
from datetime import timedelta

In [ ]:
def train_model_annually(df, start_date, end_date, window_size=30):
    models = []
    # iterate between start_date and end_date with 1 year step size
    while start_date < end_date:
        next_year = start_date + timedelta(year=1)
        if next_year > end_date:
            next_year = end_date
        final_4d_array, df_sub, stock_codes = get_pre_processed_data(df, start_date, next_year, window_size)
        model = apply_lstm_model(final_4d_array, df_sub, stock_codes)
        start_date = next_year
        models.append(model)
    return models